In [5]:
import pandas as pd
import numpy as np

In [25]:
df = pd.read_csv('data/googleplaystore.csv')
#df.head()
df['Genres'] = df.Genres.str.split(';')
#unique, counts = np.unique(df.Genres.values, return_counts=True)

In [26]:
genres = ["Action","Action & Adventure","Adventure","Art & Design","Arcade","Auto & Vehicles","Beauty","Board","Books & Reference","Brain Games","Business","Card","Casino","Casual","Comics","Communication","Creativity","Dating","Education","Educational","Entertainment","Events","Finance","Food & Drink","Health & Fitness","House & Home","Libraries & Demo","Lifestyle","Maps & Navigation","Medical","Music","Music & Audio","Music & Video","News & Magazines","Parenting","Personalization","Photography","Pretend Play","Productivity","Puzzle","Racing","Role Playing","Shopping","Simulation","Social","Sports","Strategy","Tools","Travel & Local","Trivia","Video Players & Editors","Weather","Word"]
def generate_lst(genre):
    lst_temp = []
    for i in range(len(df)):
        if genre in df.Genres[i]:
            lst_temp.append(1)
        else:
            lst_temp.append(0)
    return lst_temp
d = {genre: generate_lst(genre) for genre in genres}
transfored_genres = pd.DataFrame(d)
df = pd.concat([df,transfored_genres],axis=1)

In [27]:
df.head()

,App,Category,Rating,Reviews,Size,Installs,Type,Price,Content Rating,Genres,...,Simulation,Social,Sports,Strategy,Tools,Travel & Local,Trivia,Video Players & Editors,Weather,Word
0,Photo Editor & Candy Camera & Grid & ScrapBook,ART_AND_DESIGN,4.1,159,19M,"10,000+",Free,0,Everyone,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
1,Coloring book moana,ART_AND_DESIGN,3.9,967,14M,"500,000+",Free,0,Everyone,"[Art & Design, Pretend Play]",...,0,0,0,0,0,0,0,0,0,0
2,"U Launcher Lite – FREE Live Cool Themes, Hide ...",ART_AND_DESIGN,4.7,87510,8.7M,"5,000,000+",Free,0,Everyone,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
3,Sketch - Draw & Paint,ART_AND_DESIGN,4.5,215644,25M,"50,000,000+",Free,0,Teen,[Art & Design],...,0,0,0,0,0,0,0,0,0,0
4,Pixel Draw - Number Art Coloring Book,ART_AND_DESIGN,4.3,967,2.8M,"100,000+",Free,0,Everyone,"[Art & Design, Creativity]",...,0,0,0,0,0,0,0,0,0,0


In [10]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler

In [28]:
#unique, counts = np.unique(df.Genres.values, return_counts=True)
X = df.drop('Installs', axis=1)
y = df[['Installs']]
xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [29]:
xTrain.columns

Index(['App', 'Category', 'Rating', 'Reviews', 'Size', 'Type', 'Price',
       'Content Rating', 'Genres', 'Last Updated', 'Current Ver',
       'Android Ver', 'Action', 'Action & Adventure', 'Adventure',
       'Art & Design', 'Arcade', 'Auto & Vehicles', 'Beauty', 'Board',
       'Books & Reference', 'Brain Games', 'Business', 'Card', 'Casino',
       'Casual', 'Comics', 'Communication', 'Creativity', 'Dating',
       'Education', 'Educational', 'Entertainment', 'Events', 'Finance',
       'Food & Drink', 'Health & Fitness', 'House & Home', 'Libraries & Demo',
       'Lifestyle', 'Maps & Navigation', 'Medical', 'Music', 'Music & Audio',
       'Music & Video', 'News & Magazines', 'Parenting', 'Personalization',
       'Photography', 'Pretend Play', 'Productivity', 'Puzzle', 'Racing',
       'Role Playing', 'Shopping', 'Simulation', 'Social', 'Sports',
       'Strategy', 'Tools', 'Travel & Local', 'Trivia',
       'Video Players & Editors', 'Weather', 'Word'],
      dtype='object')

In [30]:
enc = OneHotEncoder(sparse=False)
category = enc.fit_transform(xTrain[['Category']])
category = pd.DataFrame(category, columns = enc.get_feature_names())

In [31]:
enc = OrdinalEncoder(categories = [['Everyone', 'Everyone 10+', 'Teen', 'Mature 17+', 'Adults only 18+','Unrated']])
content_rating = pd.DataFrame(enc.fit_transform(xTrain[['Content Rating']]),columns=['Content Rating'])

In [32]:
scaler = StandardScaler()
scaled_reviews = scaler.fit_transform(pd.DataFrame(xTrain['Reviews']))
scaled_reviews = pd.DataFrame(scaled_reviews,columns=['Number of Reviews'])

In [34]:
le = LabelEncoder()
transfored_target = pd.DataFrame(le.fit_transform(yTrain['Installs']),columns=['Number of Installs'])
transfored_target

,Number of Installs
0,3
1,8
2,18
3,4
4,9
5,9
6,4
7,4
8,7
9,3


In [35]:
yTrain

,Installs
6762,"1,000+"
401,"10,000,000+"
7420,500+
1631,"1,000,000+"
6475,100+
9371,100+
5689,"1,000,000+"
4382,"1,000,000+"
4317,"10,000+"
10403,"1,000+"


In [36]:
#newdf.to_excel('train_set.xlsx', sheet_name='sheet1', index=False)
#Q1: transform App?
#Q2: Rating?
#Q3: Size? Varies with device
#Q4: Installs? Ordinal, label encoder?
#Q5: Type? Free, Paid
#Q6: gitignore?
#Q7: what about the review.csv?